In [4]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
   ----- ---------------------------------- 30.7/232.6 kB 1.4 MB/s eta 0:00:01
   -------------------------------------- - 225.3/232.6 kB 4.7 MB/s eta 0:00:01
   ---------------------------------------- 232.6/232.6 kB 3.6 MB/s eta 0:00:00


In [2]:
pip install streamlit langchain openai chromadb PyPDF2


  Using cached pydantic-2.11.9-py3-none-any.whl.metadata (68 kB)
  Using cached anyio-4.11.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached uvicorn-0.37.0-py3-none-any.whl.metadata (6.6 kB)
     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ---------------------------------------- 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached typer-0.19.2-py3-none-any.whl.metadata (16 kB)
     ---------------------------------------- 0.0/43.0 kB ? e

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.15.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 6.32.1 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import os
import re
import tempfile
from typing import List, Optional

import streamlit as st
from PyPDF2 import PdfReader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

PERSIST_DIR = "./chroma_resume_db"
CHUNK_SIZE = 800
CHUNK_OVERLAP = 120
EMBEDDING_MODEL = "text-embedding-3-small"  # OpenAI embedding model name (adjustable)
LLM_MODEL = "gpt-4o-mini"  # example LLM (adjustable)

# ---------------------
# Helpers
# ---------------------

def pdf_to_text(file_path: str) -> str:
    """Extract text from PDF using PyPDF2 (works for many simple PDFs)."""
    text_chunks = []
    try:
        reader = PdfReader(file_path)
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text_chunks.append(page_text)
    except Exception as e:
        st.warning(f"PDF parsing error for {file_path}: {e}")
    return "\n".join(text_chunks)


def extract_candidate_name(text: str) -> Optional[str]:
    """Heuristic extraction of candidate name from resume text.
    Strategy:
      - Look for lines beginning with 'Name' or 'Candidate' (common headers)
      - Otherwise use the first non-empty line that looks like a name (no @, no 'linkedin', not too long)
    """
    if not text:
        return None

    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]

    # 1) look for explicit labels
    for ln in lines[:20]:
        m = re.match(r"^(?:name|candidate|full name)[:\-\s]{0,3}(.+)$", ln, flags=re.I)
        if m:
            candidate = m.group(1).strip()
            candidate = re.sub(r"[|,].*", "", candidate).strip()
            if 2 <= len(candidate) <= 60:
                return candidate

    # 2) fallback: first line that looks like a name
    for ln in lines[:10]:
        if "@" in ln or "linkedin" in ln.lower() or "www." in ln.lower() or re.search(r"\d", ln):
            continue
        # allow shortish lines with letters and spaces
        if 2 <= len(ln) <= 50 and re.match(r"^[A-Za-z .'-]+$", ln):
            return ln

    # 3) last fallback: filename or None handled outside
    return None


def build_or_load_vectorstore(embeddings: OpenAIEmbeddings, persist_dir: str = PERSIST_DIR) -> Chroma:
    """Load existing Chroma DB if present, otherwise create an empty one."""
    if os.path.exists(persist_dir) and os.listdir(persist_dir):
        vectordb = Chroma(persist_directory=persist_dir, embedding_function=embeddings)
    else:
        vectordb = Chroma(persist_directory=persist_dir, embedding_function=embeddings)
    return vectordb


def index_resumes(pdf_files: List[str], embeddings: OpenAIEmbeddings, persist_dir: str = PERSIST_DIR):
    """Parse PDFs, split into chunks, and add to Chroma with metadata (source filename, candidate_name).
    pdf_files: list of file paths
    """
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    vectordb = build_or_load_vectorstore(embeddings, persist_dir)

    added = 0
    for path in pdf_files:
        text = pdf_to_text(path)
        if not text.strip():
            st.warning(f"No text found in {os.path.basename(path)} - skipping")
            continue

        candidate_name = extract_candidate_name(text) or os.path.splitext(os.path.basename(path))[0]

        chunks = text_splitter.split_text(text)
        metadatas = [{
            "source": os.path.basename(path),
            "candidate_name": candidate_name,
        } for _ in chunks]

        vectordb.add_texts(texts=chunks, metadatas=metadatas)
        added += len(chunks)

    vectordb.persist()
    return added


# ---------------------
# Streamlit UI
# ---------------------

st.set_page_config(page_title="RAG Resume Search", layout="wide")
st.title("RAG Resume Search — LangChain + Streamlit")

if "messages" not in st.session_state:
    st.session_state["messages"] = []

# Sidebar: uploader + controls
with st.sidebar:
    st.header("Index Resumes")
    uploaded_files = st.file_uploader("Upload PDF resumes (multiple)", type=["pdf"], accept_multiple_files=True)

    build_index = st.button("Build / Update Index from uploaded files")
    clear_index = st.button("Clear stored index (delete DB)")

    st.markdown("---")
    st.header("Search Options")
    candidate_query = st.text_input("Search by candidate name (exact or partial)")
    question = st.text_input("Or ask a question about selected candidate(s) / all resumes")

    llm_temperature = st.slider("LLM temperature", 0.0, 1.0, 0.1, 0.05)

# Handle clear index
if clear_index:
    if os.path.exists(PERSIST_DIR):
        import shutil

        shutil.rmtree(PERSIST_DIR)
        st.success("Cleared persistent vectorstore")
    else:
        st.info("No persistent database found to delete")

# Initialize embeddings and vectorstore
if "embeddings" not in st.session_state:
    try:
        st.session_state["embeddings"] = OpenAIEmbeddings(model=EMBEDDING_MODEL)
    except Exception as e:
        st.error("Failed to initialize embeddings. Make sure OPENAI_API_KEY is set and langchain/openai packages installed.")
        st.stop()

embeddings = st.session_state["embeddings"]
vectordb = build_or_load_vectorstore(embeddings)

# If user asked to build index from uploaded files
if build_index:
    if not uploaded_files:
        st.warning("Please upload one or more PDF resumes first.")
    else:
        # Save uploaded files to temporary files so PyPDF2 can read them
        tmp_paths = []
        for up in uploaded_files:
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp:
                tmp.write(up.getbuffer())
                tmp_paths.append(tmp.name)

        with st.spinner("Indexing resumes — this can take a few minutes depending on how many files you uploaded"):
            added_chunks = index_resumes(tmp_paths, embeddings)
        st.success(f"Indexed {len(tmp_paths)} files, added ~{added_chunks} chunks to vectorstore")

# Candidate filter button
search_by_candidate = st.button("Find candidate")

# Prepare retriever and conversational chain
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 5})
llm = ChatOpenAI(model_name=LLM_MODEL, temperature=llm_temperature)
qa_chain = ConversationalRetrievalChain.from_llm(llm, retriever)

# If user wants to find candidate by name
selected_candidate_sources = None
if search_by_candidate:
    if not candidate_query:
        st.warning("Enter a candidate name or partial name to search by metadata")
    else:
        # use chroma's metadata filter ability via similarity_search_with_relevance_scores and filtering
        # Since langchain Chroma allows passing a filter dict to similarity_search, we'll try that
        try:
            # partial match: find any candidate_name containing the query (case-insensitive)
            # Chroma doesn't support substring filters; instead we'll fetch candidates by searching for the query as text
            # then filter by metadata client-side
            hits = vectordb.similarity_search(candidate_query, k=20)
            filtered = [doc for doc in hits if candidate_query.lower() in doc.metadata.get("candidate_name", "").lower()]

            if not filtered:
                st.info("No candidate matched by metadata. Showing top semantic matches instead.")
                filtered = hits

            # collect distinct candidate names and sources
            candidates = {}
            for d in filtered:
                key = (d.metadata.get("candidate_name"), d.metadata.get("source"))
                if key not in candidates:
                    candidates[key] = d

            st.session_state.setdefault("last_candidate_hits", [])
            st.session_state["last_candidate_hits"] = list(candidates.keys())

            st.write("Found candidates (candidate_name, source):")
            for idx, (cn, src) in enumerate(candidates.keys()):
                st.write(f"{idx+1}. {cn} — {src}")

            selected_candidate_sources = [src for (_cn, src) in candidates.keys()]
        except Exception as e:
            st.error(f"Error when searching vectorstore: {e}")

# If the user types a question or chooses a candidate, run the conversational QA
if st.button("Ask"):
    if not question:
        st.warning("Type a question in the 'Or ask a question' field first.")
    else:
        # if the user specified a candidate name, we restrict the retriever to those sources
        if candidate_query and st.session_state.get("last_candidate_hits"):
            # metadata-restricted retrieval: create a new retriever that filters by source files
            candidate_sources = [src for (_cn, src) in st.session_state["last_candidate_hits"]]

            def filtered_retriever(query):
                docs = []
                for src in candidate_sources:
                    hits = vectordb.similarity_search(query, k=5)
                    src_hits = [d for d in hits if d.metadata.get("source") == src]
                    docs.extend(src_hits)
                return docs

            # fallback: use the normal chain but with prefiltered docs passed as "retriever" is harder to adapt.
            # Simpler approach: build a temporary in-memory retriever by creating embeddings for the question and
            # performing similarity_search on the main vectordb then filtering by source.
            hits = vectordb.similarity_search(question, k=8)
            filtered_hits = [d for d in hits if d.metadata.get("source") in candidate_sources]
            if not filtered_hits:
                st.info("No matching chunks found for that candidate. Showing general results instead.")
                filtered_hits = hits

            # Prepare context text for LLM summarization
            context_text = "\n\n".join([d.page_content for d in filtered_hits[:6]])
            prompt = f"You are given the following resume excerpts:\n\n{context_text}\n\nAnswer the question concisely:\n{question}"
            response = llm.call_as_llm(prompt) if hasattr(llm, "call_as_llm") else llm.generate([prompt])

            # llm.call_as_llm compatibility handling
            if isinstance(response, str):
                answer = response
            else:
                try:
                    # langchain newer API returns an LLMResult; grab text
                    answer = response.generations[0][0].text
                except Exception:
                    answer = str(response)

            st.markdown("**Answer:**")
            st.write(answer)
        else:
            # general question across all resumes
            with st.spinner("Running RAG retrieval + LLM..."):
                result = qa_chain({"question": question, "chat_history": []})
            st.markdown("**Answer:**")
            st.write(result.get("answer"))

# Show index stats
try:
    n_docs = vectordb._collection.count()
    st.sidebar.write(f"Indexed documents (approx chunks): {n_docs}")
except Exception:
    pass

st.markdown("---")
st.write("Tips:\n- Upload PDFs and click 'Build / Update Index'.\n- Use 'Search by candidate name' to filter results by metadata.\n- Use the Ask box to query a specific candidate or all resumes.")


2025-09-26 13:57:58.248 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-26 13:57:58.250 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-26 13:57:58.330 
  command:

    streamlit run C:\Users\raghu\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-09-26 13:57:58.330 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-26 13:57:58.330 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-26 13:57:58.330 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-26 13:57:58.335 Session state does not function when running a script without `st

KeyError: 'st.session_state has no key "embeddings". Did you forget to initialize it? More info: https://docs.streamlit.io/develop/concepts/architecture/session-state#initialization'

In [4]:
pip install -U langchain langchain-community langchain-openai chromadb PyPDF2 streamlit tiktoken openai


  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
     ---------------------------------------- 0.0/76.3 kB ? eta -:--:--
     ---------------------------------------- 76.3/76.3 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   --------------- ------------------------ 1.0/2.5 MB 31.7 MB/s eta 0:00:01
   ---------------------------------------  2.5/2.5 MB 32.3 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 18.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/75.0 kB ? eta -:--:--
   ---------------------------------------- 75.0/75.0 kB 4.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/884.4 kB ? eta -:--:--
   --------------------------------------  880.6/884.4 kB 54.4 MB/s eta 0:00:01
   --------------------------------------- 884.4/884.4 kB 18.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/453.3 kB ? eta -:--:--
   -----------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.15.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 6.32.1 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
